In [1]:
# ! pip install findspark

In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col,isnan,when,count, trim
import pyspark.sql.functions as F

## создаем сессию

In [5]:
spark = (
    SparkSession
        .builder
        .appName("OTUS")
#         .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "10g")
        .config("spark.driver.memory", "10g")
        .getOrCreate()
)

## читаем данные

прочитаем только один из файлов для ускорения процесса

In [6]:
schema = StructType(
    [
        StructField("tranaction_id",IntegerType(),True),
        StructField("tx_datetime",StringType(),True),
        StructField("customer_id",IntegerType(),True),
        StructField("terminal_id",IntegerType(),True),
        StructField("tx_amount",DoubleType(),True),
        StructField("tx_time_seconds",IntegerType(),True),
        StructField("tx_time_days",IntegerType(),True),
        StructField("tx_fraud",IntegerType(),True),
        StructField("tx_fraud_scenario",IntegerType(),True),
    ]
)

In [7]:
# df = spark.read.options(header=True, delimiter=",", inferSchema=False).schema(schema).csv('/user/root/datasets/set01')

In [8]:
# df = spark.read.options(header=True, delimiter=",", inferSchema=False).schema(schema).csv(
#     [
# #         '/user/root/datasets/set01/2022-10-05.txt',
#          '/user/root/datasets/set01/2022-11-04.txt'
#     ]
# )

In [9]:
# df.write.parquet("/user/root/datasets/set01/data_part.parquet")

In [10]:
df = spark.read.options(header=True,inferSchema=False).schema(schema).parquet(
    "/user/root/datasets/set01/data_part.parquet"
)

In [11]:
df.printSchema()

root
 |-- tranaction_id: integer (nullable = true)
 |-- tx_datetime: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- terminal_id: integer (nullable = true)
 |-- tx_amount: double (nullable = true)
 |-- tx_time_seconds: integer (nullable = true)
 |-- tx_time_days: integer (nullable = true)
 |-- tx_fraud: integer (nullable = true)
 |-- tx_fraud_scenario: integer (nullable = true)



In [12]:
df.show(10)

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|   1838826044|2022-11-07 15:08:53|     850577|        324|    65.72|      101401733|        1173|       0|                0|
|   1838826045|2022-11-07 15:09:45|     850579|        734|    83.88|      101401785|        1173|       0|                0|
|   1838826046|2022-11-07 12:29:14|     850579|        533|    87.64|      101392154|        1173|       0|                0|
|   1838826047|2022-11-07 06:50:02|     850580|        826|     3.81|      101371802|        1173|       0|                0|
|   1838826048|2022-11-07 11:08:58|     850581|        115|    50.42|      101387338|        1173|       1|           

## какие есть таргеты?

In [13]:
# df.select('tx_fraud').distinct().limit(20).collect()

In [14]:
# df.select('tx_fraud_scenario').distinct().show()

In [15]:
df.groupBy("tx_fraud").count().show()

+--------+--------+
|tx_fraud|   count|
+--------+--------+
|       1| 1406151|
|       0|45592832|
+--------+--------+



In [16]:
df.groupBy("tx_fraud_scenario").count().show()

+-----------------+--------+
|tx_fraud_scenario|   count|
+-----------------+--------+
|                1|   26226|
|                3|   50957|
|                2| 1328968|
|                0|45592832|
+-----------------+--------+



## общее количество

In [17]:
df.count()

46998983

In [18]:
# N = 1879794138
N = 46998983

## удалим пропуски по колонкам

In [19]:
df2 = df.select(
    [
        count(
            when(
                (col(c) == '' ) |
                col(c).isNull() |
                isnan(c),
                c
            )
        ).alias(c) 
        for c in df.columns
    ]
)

In [20]:
df2.show()

+-------------+-----------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-----------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|            0|          0|          0|       2298|        0|              0|           0|       0|                0|
+-------------+-----------+-----------+-----------+---------+---------------+------------+--------+-----------------+



In [21]:
def remove_missing(df_):
    start_size = df_.count()
    df_ = df_.dropna(how="any")
    size = df_.count()
    print(f"Было удалено {start_size - size} элементов как пропусков")
    return df_

## удалим дубликаты

In [22]:
duplicate_rows = df.count() - df.dropDuplicates().count()
print(duplicate_rows)

8


In [23]:
def remove_duplcates(df_):
    start_size = df_.count()
    df_ = df_.dropDuplicates()
    size = df_.count()
    print(f"Было удалено {start_size - size} элементов как дубликатов")
    return df_

## удалим выбросы

In [24]:
def remove_outliers(df_, col_name):
    start_size = df_.count()
    Q1 = df_.approxQuantile(col_name, [0.25], 0.05)[0]
    Q3 = df_.approxQuantile(col_name, [0.75], 0.05)[0]
    IQR = Q3 - Q1
    ub = Q3 + 1.5 * IQR
    df_ = df_.filter(F.col(col_name) <= ub)
    size = df_.count()
    print(f"Для колонки {col_name} было удалено {start_size - size} элементов как выбросов")
    return df_

## проверка колонки tx_datetime на корректность формата

In [25]:
def remove_incorrect_date(df_):
    start_size = df_.count()
    date_pattern = "^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"
    df_ = df_.filter(F.col("tx_datetime").rlike(date_pattern))
    size = df_.count()
    print(f"Было удалено {start_size - size} с датой в неправильном формате")
    return df_

## очистим данные

In [26]:
def cleanse_data(df_):
    start_size = df_.count()
    df_ = remove_duplcates(df_)
    df_ = remove_incorrect_date(df_)
    df_ = remove_missing(df_)
    for col_name in ["tx_amount", "terminal_id", "tx_time_seconds", "tx_time_days"]:
        df_ = remove_outliers(df_, col_name)
    size = df_.count()
    print(f"После очистки из {start_size} строк осталось {size}")
    return df_

In [27]:
%%time
df_cleansed = cleanse_data(df)

Было удалено 8 элементов как дубликатов
Было удалено 0 с датой в неправильном формате
Было удалено 2298 элементов как пропусков
Для колонки tx_amount было удалено 1287248 элементов как выбросов
Для колонки terminal_id было удалено 15859 элементов как выбросов
Для колонки tx_time_seconds было удалено 0 элементов как выбросов
Для колонки tx_time_days было удалено 0 элементов как выбросов
После очистки из 46998983 строк осталось 45693570
CPU times: user 123 ms, sys: 41.3 ms, total: 164 ms
Wall time: 16min 41s


In [28]:
df_cleansed.write.parquet("/user/root/datasets/set01/data_cleansed.parquet")

In [29]:
df_cleansed.write.parquet("s3a://mlops-hw3-vos/data_cleansed.parquet",mode="overwrite")